In [1]:
import pandas as pd
from pprint import pprint
from collections import namedtuple
import numpy as np

In [2]:
df19_20_raw = pd.read_csv('base.csv', sep=';', encoding='latin-1')
df20_21_raw = pd.read_csv('base20-21_latin1.csv', sep=';', encoding='latin-1')
df19_20 = df19_20_raw.copy()
df20_21 = df20_21_raw.copy()

In [3]:
def ntuples(df):
    """ Facilita o autocomplete de nomes de colunas """
    list_of_names = df.columns.values
    list_of_names_dict = {x: x for x in list_of_names}

    Varnames = namedtuple('Varnames', list_of_names)
    return Varnames(**list_of_names_dict)

### Remoção de linhas com valores nulos

In [4]:
df19_20.dropna(inplace=True)
df20_21.dropna(inplace=True)

### Renomeação das colunas

In [5]:
def rename_columns(df):
    df.columns = df.columns.str.lower().str.strip('\n')
    rename = {
        'funcionário - código': 'cod_funcionario',
        'classe de manutenção - código': 'cod_classe_manutencao',
        'sistema veicular - código': 'cod_sistema',
        'subsistema veicular - código': 'cod_subsistema',
        'data de entrada': 'data_entrada',
        'data de saída': 'data_saida',
        'horas': 'horas',
        '%sistema veicular': '%sistema_veicular',
        '%funcionário': '%funcionario',
        '%data de entrada': '%data_entrada',
        '%data de saída': '%data_saida',
        '%classe de manutenção': '%classe',
        '% geral': '%geral'

    }
    df = df.rename(rename, axis=1)
    df = df[['cod_classe_manutencao', 'cod_sistema', 'cod_subsistema', 'cod_funcionario', 'data_entrada', 'data_saida', 'horas']]
    return df


# df['data_entrada'] = pd.to_datetime(df['data_entrada'])#.dt.strftime('%d/%m/%Y')
# df['data_saida'] = pd.to_datetime(df['data_saida'])#.dt.strftime('%d/%m/%Y')
df19_20 = rename_columns(df19_20)
df20_21 = rename_columns(df20_21)
c = ntuples(df20_21)

### Conserto de datas

In [6]:
def fix_dates(dfr, date_column1, date_column2):
    df = dfr.copy()
    for col in [date_column1, date_column2]:
        try:
            df[col] = pd.to_datetime(df[col], format='%m/%d/%Y')
        except ValueError:
            df[col] = pd.to_datetime(df[col], format='%d/%m/%Y')

    original_row_count = len(df[date_column1])

    to_drop = pd.to_datetime(df[date_column1]) > pd.to_datetime(df[date_column2])
    df = df[~to_drop]
    print(f'Dropped {original_row_count - len(df[date_column1])} invalid rows')

    return df

In [7]:
df20_21 = fix_dates(df20_21, c.data_entrada, c.data_saida)
df19_20 = fix_dates(df19_20, c.data_entrada, c.data_saida)

Dropped 418 invalid rows
Dropped 380 invalid rows


### Merge dos arquivos (2019-2021)

In [8]:
merged_df = pd.concat([df19_20, df20_21])
merged_df = merged_df.reset_index().drop('index', axis=1)
merged_df

,cod_classe_manutencao,cod_sistema,cod_subsistema,cod_funcionario,data_entrada,data_saida,horas
0,29,1400,1402,22955,2020-04-01,2020-04-01,2.35
1,29,1300,1312,30413,2020-04-01,2020-04-01,4.74
2,29,1400,1410,41598,2020-04-01,2020-04-01,2.99
3,29,1000,1003,50082,2020-04-01,2020-04-01,0.29
4,29,1300,1301,52596,2020-04-01,2020-04-01,3.35
...,...,...,...,...,...,...,...
110570,38,400,424,110209,2021-03-31,2021-04-02,2.71
110571,38,200,208,57143,2021-03-31,2021-04-05,5.77
110572,38,200,208,94070,2021-03-31,2021-04-05,5.77
110573,38,200,208,108106,2021-03-31,2021-04-05,2.63


### Mais linhas estranhas

In [9]:
merged_df = merged_df[merged_df[c.data_entrada].dt.year >= 2019]
merged_df = merged_df[merged_df[c.horas] > 0]

#### Detecção & remoção de outliers

In [10]:
def is_outlier(column, deviations):
    lower = column.mean() - (column.std()*deviations)
    upper = column.mean() + (column.std()*deviations)
    return ~column.between(lower, upper)


def remove_outliers(df, deviations=3):
    w_outliers = len(df)
    df = df[~df.groupby([c.cod_sistema, c.cod_subsistema])[c.horas].apply(is_outlier, deviations=deviations)]

    print(f'Bruto\tSem Outlier\n{w_outliers} vs {len(df)}')
    print(f'Diferença: {w_outliers - len(df)}')

    return df

In [11]:
merged_df = remove_outliers(merged_df, 2)

Bruto	Sem Outlier
108083 vs 103917
Diferença: 4166


#### Remoção de valores nulos (= 0)

In [12]:
def remove_zero_values(df):
    b4_remove0 = len(df)
    df = df.query('horas != 0')
    print(f'Removidas {b4_remove0 - len(df)} linhas')
    return df


merged_df = remove_zero_values(merged_df)

Removidas 0 linhas


#### Remoção de linhas com horas > 24, mas saida - entrada = 1

In [44]:
def remove_days_more24h(df, date_column1, date_column2, hour_column):
    over24h = df[hour_column] > 24
    same_day = (df[date_column1] - df[date_column2]).dt.components.days == 0
    df = df[~(over24h & same_day)]
    return df


merged_df = remove_days_more24h(merged_df, c.data_entrada, c.data_saida, c.horas)

---

### Exportação .csv

In [46]:
merged_df.to_csv('base19-21.csv', sep=';', encoding='latin-1', index=False)